In [1]:
!pip install pycaret

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import random
import os
import pandas as pd
import numpy as np
import datetime
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from pycaret.regression import *


In [2]:
# 데이터 로드
train_data = pd.read_csv('data/train.csv').drop(columns=['ID', '제품'])

# holiday effect df 만들기 (위에서 이미 수행한 내용을 그대로 사용)

# holiday effect df만들기

from datetime import datetime, timedelta

start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 4, 4)

date_list = []
day_type_list = []

current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    day_type_list.append(1 if current_date.weekday() in [5, 6] else 0)
    current_date += timedelta(days=1)

# Create a DataFrame
date_df = pd.DataFrame({'Date': date_list, 'Day_Type': day_type_list})

print(date_df.tail())

           Date  Day_Type
454  2023-03-31         0
455  2023-04-01         1
456  2023-04-02         1
457  2023-04-03         0
458  2023-04-04         0


In [3]:
holiday = train_data.copy()

In [4]:
holiday.drop(['브랜드', '대분류','중분류','소분류'], axis = 1, inplace = True)

In [5]:
i = 0
for date in holiday:
    holiday[date] = day_type_list[i]
    i +=1
    if i  > len(day_type_list):
        break
        

In [6]:
combined_df = pd.merge(train_data, holiday,left_index = True, right_index = True, how = 'left')
combined_df

,대분류,중분류,소분류,브랜드,2022-01-01_x,2022-01-02_x,2022-01-03_x,2022-01-04_x,2022-01-05_x,2022-01-06_x,...,2023-03-26_y,2023-03-27_y,2023-03-28_y,2023-03-29_y,2023-03-30_y,2023-03-31_y,2023-04-01_y,2023-04-02_y,2023-04-03_y,2023-04-04_y
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15886,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15887,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15888,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0


In [7]:
train_data = combined_df.copy()

In [8]:
# Label Encoding (위에서 이미 수행한 내용을 그대로 사용)

# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [9]:
train_data

,대분류,중분류,소분류,브랜드,2022-01-01_x,2022-01-02_x,2022-01-03_x,2022-01-04_x,2022-01-05_x,2022-01-06_x,...,2023-03-26_y,2023-03-27_y,2023-03-28_y,2023-03-29_y,2023-03-30_y,2023-03-31_y,2023-04-01_y,2023-04-02_y,2023-04-03_y,2023-04-04_y
0,1,6,37,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
1,2,7,43,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
2,2,7,43,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
3,2,7,43,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
4,0,0,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,2,7,41,3169,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15886,2,7,43,3169,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15887,2,7,43,3169,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15888,2,7,43,3169,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0


In [10]:
train_data.columns[4:]

Index(['2022-01-01_x', '2022-01-02_x', '2022-01-03_x', '2022-01-04_x',
       '2022-01-05_x', '2022-01-06_x', '2022-01-07_x', '2022-01-08_x',
       '2022-01-09_x', '2022-01-10_x',
       ...
       '2023-03-26_y', '2023-03-27_y', '2023-03-28_y', '2023-03-29_y',
       '2023-03-30_y', '2023-03-31_y', '2023-04-01_y', '2023-04-02_y',
       '2023-04-03_y', '2023-04-04_y'],
      dtype='object', length=918)

In [ ]:
# [Hyperparameter Setting]
CFG = {
    'TRAIN_WINDOW_SIZE': 90,  # 90일치로 학습
    'PREDICT_SIZE': 21,  # 21일치 예측
    'EPOCHS': 10,
    'LEARNING_RATE': 1e-4,
    'BATCH_SIZE': 1024,
    'SEED': 41,
    'HIDDEN_SIZE': 512,  # Hidden size of GRU model
}

In [ ]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량, 브랜드, 그리고 브랜드 키워드 카운트 정보가 있는 데이터 프레임
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    holiday_start_index = data.columns.get_loc('2022-01-01_y')  # Find the index of the first '2022-01-01_y' column
    window_size = train_size + predict_size

    input_data = np.empty((num_rows * (459 - window_size + 1), train_size, 6)) # 6 features: 대분류, 중분류, 소분류, 브랜드, sales. holiday
    target_data = np.empty((num_rows * (459 - window_size + 1), predict_size))



    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])  # Extract encoding info ('대분류', '중분류', '소분류', '브랜드')
        sales_data = np.array(data.iloc[i, 4: holiday_start_index-1])
        holiday_data = np.array(data.iloc[i, holiday_start_index:])


        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size], holiday_data[j : j + train_size]))
            input_data[i * (459 - window_size + 1) + j] = temp_data
            target_data[i * (459 - window_size + 1) + j] = window[train_size:]


    return input_data, target_data

In [ ]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량, 브랜드, 그리고 holiday 정보가 있는 데이터 프레임
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    holiday_start_index = data.columns.get_loc('2022-01-01_y')  # Find the index of the first '2022-01-01_y' column
    
    input_data = np.empty((num_rows, train_size, 6))
    

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])  # Extract encoding info ('대분류', '중분류', '소분류', '브랜드')
        sales_data = np.array(data.iloc[i, holiday_start_index-91:holiday_start_index-1])  # Extract daily sales data for the past 'train_size' days
        holiday_data = np.array(data.iloc[i, -train_size:])  # Extract holiday data for the past 'train_size' days

        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size], holiday_data))
        #temp_data = np.column_stack((encode_info, sales_data, holiday_data))
        input_data[i] = temp_data
    
    return input_data


In [ ]:
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

In [ ]:
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [ ]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

In [ ]:

# 데이터 전처리 및 모델링 설정
exp_reg = setup(data=train_data, target=train_data.columns[4:], session_id=42,
                categorical_features=['대분류', '중분류', '소분류', '브랜드'],
                train_size=CFG['TRAIN_WINDOW_SIZE'], fold_strategy='timeseries')


# 모델 훈련 및 비교 (여기서는 최고 5개 모델을 선택)
top_models = compare_models(n_select=5)

# 모델 튜닝 (하이퍼파라미터 튜닝)
tuned_models = [tune_model(model) for model in top_models]

# 모델 앙상블
ensemble_model = blend_models(tuned_models)


# 모델 예측
test_data = make_predict_data(train_data)
predictions = predict_model(ensemble_model, data=test_data)

# 예측 결과 확인
predictions.head()

# 예측 결과를 원하는 형태로 변환 (아래는 기존 코드와 호환성을 유지하기 위한 변환)
for idx in range(len(predictions)):
    predictions.iloc[idx, :] = predictions.iloc[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# Post-processing
predictions = predictions.round(0).astype(int)

# Submission 파일 생성
submit = pd.read_csv('data/sample_submission.csv')
submit.iloc[:, 1:] = predictions
submit.to_csv('data/pycaret_stacked_gru_weekend_submit.csv', index=False)
